In [1]:
from data.helper import *

In [2]:
DF = get_plan_df("pfcalcul")

100%|██████████| 4800/4800 [00:00<00:00, 7543.77it/s]


Error parsing /home/dzc/code/work/goose/experiments/_log_plan/pfcalcul/spanner_wl_none_equivalent-weighted_rank-svm_plan_fd_2_0_09_0.log: 'NoneType' object has no attribute 'group'


In [3]:
COLUMNS = [
    "facts",
    "optimiser",
    "features",
    "iterations",
    "feature_pruning",
    # "data_pruning",
]

def get_df_copy():
    df = DF.copy()
    # df = df[df.optimiser == "svr"]
    # df = df[df.iterations == "4"]
    df = df[df.facts == "fd"]
    df = df[df.features == "wl"]
    df = df[df.data_pruning == "none"]
    # df = df[df.feature_pruning == "none"]
    return df

for metric in [
    # "tried",
    "solved",
]:
    print(metric)
    df = get_df_copy()
    # combine rows with same pruning, optimiser and iterations, and sum the solved, with sum values made into int
    df = df.groupby(COLUMNS + ["domain"]).sum().reset_index()
    df = df[COLUMNS + ["domain", metric]]
    # pivot table
    df = df.pivot_table(index=COLUMNS, columns="domain", values=metric)
    # add a column that sums all integer entries per row
    df["total"] = df.sum(axis=1)
    # make entries int
    df = df.astype(int)
    # df = df.fillna(0)
    # replace 0 with NaN
    df = df.replace(0, -1)
    # display df without hiding rows
    with pd.option_context("display.max_rows", None):
        display(df)

solved


,,,,domain,total
facts,optimiser,features,iterations,feature_pruning,


In [4]:
for metric in ["expanded", "runtime", "plan_length"]:
    htmls = ""
    for domain in DOMAINS:
        df = get_df_copy()
        df = df[df.domain == domain]
        # replace -1 values with Nan
        df = df.replace(-1, np.nan)
        # print(len(df))
        df["config"] = df["features"] + " - " + df["iterations"] + " - " + df["optimiser"]
        fig = px.line(
            df,
            x="problem",
            y=metric,
            color="config",
            symbol="facts",
            line_dash="facts",
            title=f"{domain} {metric}",
            width=800,
            height=600,
            log_y=True,
            markers=True,
        )
        htmls += fig.to_html()
    with open(f"{PLOT_DIR}/{metric}.html", "w") as f:
        f.write(htmls)